In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**We'll be using CNN to implement our model.**

In [ ]:
pip install fastai==1.0.61

In [ ]:
# Importing Libraries
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers, losses
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
import pickle
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from fastai import *
from fastai.imports import *
from fastai.vision import *
from fastai.metrics import error_rate
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

np.random.seed(42)

In [ ]:
# Base Path for all files
base_dir = '../input/nike-adidas-shoes-for-image-classification-dataset/train'

**DATA LOADING FOR TRAINING**

SETTING PATH

In [ ]:
x  = '../input/nike-adidas-and-converse-imaged/train'
path = Path(x)
path.ls()

validation percentage will be 20%

In [ ]:
np.random.seed(40)
data =  ImageDataBunch.from_folder(path, train = '.', valid_pct=0.2,
                                  ds_tfms=get_transforms(), size=224,
                                  num_workers=4).normalize(imagenet_stats)

Our image dataset is stored as .jpg files in 2 different folders, with each folder bearing the name of model of the images contained in the folder. We use the ImageDataBunch.from_folder() function to load the images and assign labels the images based on the name of the folder they’re read from.

In [ ]:
data.show_batch(rows=3, figsize=(7,6),recompute_scale_factor=True)

In [ ]:
data

data.classes= This gives the classes present in our training dataset

In [ ]:
print(data.classes)
len(data.classes)
data.c

We now use a pre-trained ResNet18 Convolutional Neural Net model, and use transfer learning to learn weights of only the last layer of the network.
Why Transfer learning? Because with transfer learning, you begin with an existing (trained) neural network used for image recognition — and then tweak it a bit (or more) here and there to train a model for your particular use case. And why do we do that? Training a reasonable neural network would mean needing approximately 300,000 image samples, and to achieve really good performance, we’re going to need at least a million images.
In our case, we have approximately 711 images in our training set — you have one guess to decide if that would have been enough if were to train a neural net from scratch.
We use the create_cnn() function for loading a pre-trained ResNet18 network, that was trained on around a million images from the ImageNet database.

In [ ]:
learn = cnn_learner(data, models.resnet18, metrics=[accuracy], model_dir = Path('../kaggle/working'),path = Path("."))

FINDING LR

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestions=True)

**TRAINING MODEL**

In [ ]:
lr1 = 1e-3
lr2 = 1e-1
learn.fit_one_cycle(40,slice(lr1,lr2))

**HYPER PARAMETER TUNING**

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(20,slice(1e-4,1e-3))

In [ ]:
learn.recorder.plot_losses()

**INTERPRETTING THE RESULT**

Model performance can be validated in different ways. One of the popular methods is using the confusion matrix. Diagonal values of the matrix indicate correct predictions for each class, whereas other cell values indicate a number of wrong predictions.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(3,figsize = (25,5))

**PREDICTION USING TRAINED MODEL**

In [ ]:
img = open_image('../input/nike-adidas-and-converse-imaged/test/adidas/14.jpg')
print(learn.predict(img)[0])